In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R4HYY7NX8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-141708
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-141708


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = "udacity-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Assigning existing cluster...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=0, max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    print('Create compute cluster...')

compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())


Create compute cluster...
Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-31T19:40:35.026000+00:00', 'errors': None, 'creationTime': '2021-03-31T19:40:32.209034+00:00', 'modifiedTime': '2021-03-31T19:40:48.153914+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C' : uniform(0.5, 1.5),
        '--max_iter': choice(50, 100, 150, 200)
    }
)
# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
est = SKLearn(source_directory = os.path.join( "./" ),
        entry_script="train.py",
        compute_target=compute_target
    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        estimator=est,
        policy=policy,
        max_total_runs=16,
        max_concurrent_runs=4
    )


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c4565f22-fe1e-474e-9ad0-fdc2f28f61ff
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c4565f22-fe1e-474e-9ad0-fdc2f28f61ff?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-141708/workspaces/quick-starts-ws-141708

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-31T19:41:33.530545][API][INFO]Experiment created<END>\n""<START>[2021-03-31T19:41:33.974926][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-31T19:41:34.300821][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-31T19:41:36.1710439Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_c4565f22-fe1e-474e-9ad0-fdc2f28f61ff
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_c4565f22-fe1e-474e-9ad0-fdc2f28f61ff?wsid=/subscriptions/9a751

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metrics:", best_run.get_metrics())
print("Best run details:", best_run.get_details())
print("Best run file names:", best_run.get_file_names())



Best run metrics: {'Regularization Strength:': 0.7989305637484468, 'Max iterations:': 150, 'Accuracy': 0.9072837632776934}
Best run details: {'runId': 'HD_c4565f22-fe1e-474e-9ad0-fdc2f28f61ff_0', 'target': 'udacity-cluster', 'status': 'Completed', 'startTimeUtc': '2021-03-31T19:45:19.853959Z', 'endTimeUtc': '2021-03-31T19:47:09.558505Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'e3e64b3b-7de4-4b52-91bf-af6d316db230', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.7989305637484468', '--max_iter', '150'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'udacity-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'priority': No

In [6]:
# Save the best model
joblib.dump(value=best_run.get_details()['runDefinition']['arguments'], filename=os.path.join('./outputs', 'best_model.joblib'))


['./outputs/best_model.joblib']

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [8]:
from azureml.core import Dataset
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)
x['y'] = y
os.mkdir("./data")
x.to_csv('./data/ds_cleaned.csv')
datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')
ds_cleaned = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/ds_cleaned.csv'))])

Uploading an estimated of 1 files
Uploading data/ds_cleaned.csv
Uploaded data/ds_cleaned.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds_cleaned,
    label_column_name='y',
    compute_target=compute_target,
    n_cross_validations=5)

In [10]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()


Running on remote.


{'runId': 'AutoML_0cbe7f3a-5d27-493b-984b-14f4478d7a0c',
 'target': 'udacity-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-31T19:58:36.26686Z',
 'endTimeUtc': '2021-03-31T20:45:06.692416Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'udacity-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-141708","workspace_name":"quick-starts-ws-141708","region":"southcentralus","compute_target":"udacity-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":n

In [14]:
# Retrieve and save best automl model.
#from azureml.automl.runtime.onnx_convert import OnnxConverter
#best_aml_model, onnx_model = remote_run.get_output(return_onnx_model=True)
#OnnxConverter.save_onnx_model(onnx_model, "./automl_model.onnx")

### YOUR CODE HERE ###
best_run, best_model = remote_run.get_output()
print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs')
registered_model.download(target_dir='./outputs', exist_ok=True)


In [15]:
# Delete Compute Cluster
compute_target.delete()


Current provisioning state of AmlCompute is "Deleting"

